In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df_steam_games = pd.read_parquet('../Data/df_steam_games.parquet')
df_user_items = pd.read_parquet('../Data/df_user_items.parquet')

In [4]:
df_steam_games = df_steam_games.rename(columns={'id': 'item_id'})

In [4]:
# Crear un DF con las columnas necesarias
df_games_specs = df_steam_games.loc[:, ["specs", "item_id", "title"]]

# Analisis de los datos nulos
df_games_specs.isnull().mean()

specs      0.021270
item_id    0.000065
title      0.062866
dtype: float64

In [5]:
df_games_specs.head()

,specs,item_id,title
0,[Single-player],761140,Lost Summoner Kitty
1,"[Single-player, Multi-player, Online Multi-Pla...",643980,Ironbound
2,"[Single-player, Multi-player, Online Multi-Pla...",670290,Real Pool 3D - Poolians
3,[Single-player],767400,弹炸人2222
4,"[Single-player, Full controller support, HTC V...",773570,None


In [13]:
# Eliminar los datos nulos
df_games_specs = df_games_specs.dropna()

# Limpiar la columna "specs"
df_games_specs['specs'] = df_games_specs['specs'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ''))

In [14]:
# Crear una instancia de CountVectorizer
cv = CountVectorizer()

# Ajustar y transformar el texto en una matriz dispersa
vector = cv.fit_transform(df_games_specs['specs']).toarray()

In [9]:
# Calcular la similitud del coseno entre los valores de la columna "specs"
similarity = cosine_similarity(vector)

In [16]:
def recomendacion_juego(id_de_producto):
    # Buscar el indice ingresado en el DF "df_games_specs"
    item_id = df_games_specs[df_games_specs["item_id"] == id_de_producto].index[0]
    
    # Similitud entre el juego de entrada y todos los demás juegos en el conjunto de datos
    similar_scores = similarity[item_id]
    
    # Ordenar de manera descendente y seleccionar los cinco juegos más similares
    similar_items = sorted(list(enumerate(similar_scores)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Devolver la lista de títulos recomendados
    recommended_titles = [df_games_specs.iloc[e[0]]['title'] for e in similar_items]
    
    return recommended_titles